# Introduction to Machine Learning - Task 3

Group name: Cbbayes

Team members: mcolomer (mcolomer@student.ethz.ch), pratsink (pratsink@student.ethz.ch) and scastro (scastro@student.ethz.ch)

Spring 2021

## Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

## Preprocessing: one-hot encoding of the sequences

In [2]:
# Indicate the path to the data file
path = "../data/"

train_data = pd.read_csv(path+"train.csv")
test_data = pd.read_csv(path+"test.csv")

aminoacids = ["R", "H", "K","D","E", "S","T","N","Q","C","U",
             "G","P","A","I","L","M","F","W","Y","V"]

charge = [1,1,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
polarity=[1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
special=[0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0]


#Preprocessing training dataset

X_train = train_data["Sequence"].str.split("", n = 4, expand = True)

def map_charge(aa, charge, aminoacids):
    i = aminoacids.index(aa)
    return charge[i]

def map_polarity(aa, polarity, aminoacids):
    i = aminoacids.index(aa)
    return polarity[i]

for i in [1,2,3,4]:
    train_data["pos"+str(i)] = X_train[i]
    train_data = pd.concat([train_data,pd.get_dummies(train_data["pos"+str(i)], prefix="pos"+str(i))],axis=1)
    train_data["charge"+str(i)] = train_data["pos"+str(i)].apply(lambda aa: map_charge(aa, charge, aminoacids))
    train_data["polarity"+str(i)] = train_data["pos"+str(i)].apply(lambda aa: map_polarity(aa, polarity, aminoacids))
    train_data = train_data.drop(columns=["pos"+str(i)])
    
X_train = train_data.iloc[:, 2:90].values 
y_train = train_data.iloc[:,1].values 

#Preprocessing test dataset

X_test = test_data["Sequence"].str.split("", n = 4, expand = True)


for i in [1,2,3,4]:
    test_data["pos"+str(i)] = X_test[i]
    test_data = pd.concat([test_data,pd.get_dummies(test_data["pos"+str(i)], prefix="pos"+str(i))],axis=1)
    test_data["charge"+str(i)] = test_data["pos"+str(i)].apply(lambda aa: map_charge(aa, charge, aminoacids))
    test_data["polarity"+str(i)] = test_data["pos"+str(i)].apply(lambda aa: map_polarity(aa, polarity, aminoacids))
    test_data = test_data.drop(columns=["pos"+str(i)])
    
X_test = test_data.iloc[:, 1:89].values 


In [3]:
print(len(charge))
print(len(polarity))
print(len(special))

21
21
21


## Classification

Class are quite unbalanced:

In [4]:
print(len(y_train))
print(sum(y_train))

112000
4213


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [250,300]
    
}
# Create a based model
rf = MLPClassifier(class_weight='balanced')
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring="f1")


grid_search.fit(X_train, y_train)
grid_search.best_params_


Fitting 3 folds for each of 2 candidates, totalling 6 fits


{'n_estimators': 300}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [250,300]
    
}
# Create a based model
rf = RandomForestClassifier(class_weight='balanced_subsample')
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring="f1")


grid_search.fit(X_train, y_train)
grid_search.best_params_


In [6]:
grid_search.scorer_

make_scorer(f1_score, average=binary)

In [7]:
grid_search.best_score_

0.8120560712110794

In [10]:
print(grid_search.cv_results_)

{'mean_fit_time': array([102.68404261,  88.888105  ]), 'std_fit_time': array([ 0.93579585, 20.83585741]), 'mean_score_time': array([4.02074361, 2.90831836]), 'std_score_time': array([0.14942299, 1.18479463]), 'param_n_estimators': masked_array(data=[250, 300],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_estimators': 250}, {'n_estimators': 300}], 'split0_test_score': array([0.81603774, 0.81566456]), 'split1_test_score': array([0.80522565, 0.80524642]), 'split2_test_score': array([0.81237759, 0.81394438]), 'mean_test_score': array([0.81121366, 0.81161845]), 'std_test_score': array([0.00449009, 0.0045601 ]), 'rank_test_score': array([2, 1], dtype=int32)}


In [8]:
print(grid_search.cv_results_)

{'mean_fit_time': array([113.485286  , 102.87980874]), 'std_fit_time': array([ 1.19609629, 31.53070118]), 'mean_score_time': array([4.40012074, 3.58625555]), 'std_score_time': array([0.33826858, 0.08746272]), 'param_n_estimators': masked_array(data=[250, 350],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_estimators': 250}, {'n_estimators': 350}], 'split0_test_score': array([0.81510622, 0.81469775]), 'split1_test_score': array([0.80366388, 0.80381255]), 'split2_test_score': array([0.81739812, 0.81576893]), 'mean_test_score': array([0.81205607, 0.81142641]), 'std_test_score': array([0.00600749, 0.00540154]), 'rank_test_score': array([1, 2], dtype=int32)}


In [64]:
def fit_RFC(X_train, y_train, X_test):
    """Random forest for predicting
    the regression features"""
    clf = RandomForestClassifier(n_estimators=250, class_weight='balanced')
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    return predicted

In [ ]:
rf_random.best_params_

In [42]:
y_test = fit_RFC(X_train, y_train, X_test)

In [37]:
print(len(y_test))
print(sum(y_test))

48000
1382


In [65]:
pd.DataFrame(y_test).to_csv('output_RFC_250estim_balanced_charge_polarity.csv', index = False, header = False)             ## Save the results dataframe to a csv